# 1

## One-to-Many Sequence Problems

In [ ]:
"""
One-to-many sequence problems are the type of sequence problems where input data has one time-step and the output contains a vector of multiple
values or multiple time-steps.
"""

X = list()
Y = list()
X = [x+3 for x in range(-2, 43, 3)]

for i in X:
    output_vector = list()
    output_vector.append(i+1)
    output_vector.append(i+2)
    Y.append(output_vector)

print(X)
print(Y)

"""
[1, 4, 7, . . . . . ]
[[2, 3], [5, 6], [8, 9], . . . . ]

input contains 15 samples with one time-step and one feature value. For each value in the input sample, the corresponding output vector
contains the next two integers. For instance, if the input is 4, the output vector will contain values 5 and 6.
"""
X = np.array(X).reshape(15, 1, 1)
Y = np.array(Y)

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(1, 1)))
model.add(Dense(2))
model.compile(optimizer='adam', loss='mse')
model.fit(X, Y, epochs=1000, validation_split=0.2, batch_size=3)

test_input = array([10])
test_input = test_input.reshape((1, 1, 1))
test_output = model.predict(test_input, verbose=0)
print(test_output)

## Stacked LSTM

In [ ]:
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(1, 1)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(2))
model.compile(optimizer='adam', loss='mse')
history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1, batch_size=3)

test_output = model.predict(test_input, verbose=0)
print(test_output)

## Bidirectional LSTM

In [ ]:
from keras.layers import Bidirectional

model = Sequential()
model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(1, 1)))
model.add(Dense(2))
model.compile(optimizer='adam', loss='mse')

history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1, batch_size=3)
test_output = model.predict(test_input, verbose=0)
print(test_output)

# 2

## One-to-Many Sequence Problems with Multiple Features

In [2]:
nums = 25

X1 = list()
X2 = list()
X = list()
Y = list()

X1 = [(x+1)*2 for x in range(25)]
X2 = [(x+1)*3 for x in range(25)]

for x1, x2 in zip(X1, X2):
    output_vector = list()
    output_vector.append(x1+1)
    output_vector.append(x2+1)
    Y.append(output_vector)

X = np.column_stack((X1, X2))
print(X)

"""
[[ 2  3]
 [ 4  6]
 [ 6  9]
 .
 .]

input time-step consists of two features. The output will be a vector which contains the next two elements that correspond to 
the two features in the time-step of the input sample
"""

X = np.array(X).reshape(25, 1, 2)
Y = np.array(Y)

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(1, 2)))
model.add(Dense(2))
model.compile(optimizer='adam', loss='mse')
model.fit(X, Y, epochs=1000, validation_split=0.2, batch_size=3)

test_input = array([40, 60])
test_input = test_input.reshape((1, 1, 2))
test_output = model.predict(test_input, verbose=0)
print(test_output)

## Stacked LSTM

In [ ]:
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(1, 2)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(2))
model.compile(optimizer='adam', loss='mse')
history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1, batch_size=3)

test_input = array([40, 60])
test_input = test_input.reshape((1, 1, 2))
test_output = model.predict(test_input, verbose=0)
print(test_output)

## Bidirectional LSTM

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(1, 2)))
model.add(Dense(2))
model.compile(optimizer='adam', loss='mse')

history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1, batch_size=3)
test_output = model.predict(test_input, verbose=0)
print(test_output)

# 2

## Encoder-Decoder Model

In [ ]:
"""
To solve such sequence problems, the encoder-decoder model has been designed. The encoder-decoder model is basically a fancy name for 
neural network architecture with two LSTM layers.
The first layer works as an encoder layer and encodes the input sequence. The decoder is also an LSTM layer, which accepts three inputs: 
the encoded sequence from the encoder LSTM, the previous hidden state, and the current input. During training the actual output at 
each time-step is used to train the encoder-decoder model. While making predictions, the encoder output, the current hidden state, 
and the previous output is used as input to make predictions at each time-step. These concepts will become more understandable 
when you will see them in action in an upcoming section.
"""

## Many-to-Many Sequence Problems with Single Feature

In [ ]:
X = list()
Y = list()
X = [x for x in range(5, 301, 5)]
Y = [y for y in range(20, 316, 5)]

X = np.array(X).reshape(20, 3, 1)
Y = np.array(Y).reshape(20, 3, 1)

"""
The input X contains 20 samples where each sample contains 3 time-steps with one feature.
[[[  5]
  [ 10]
  [ 15]]
You can see that the input sample contains 3 values that are basically 3 consecutive multiples of 5.
[[[ 20]
  [ 25]
  [ 30]]

The output contains the next three consecutive multiples of 5.

For the encoder-decoder model, the output should also be converted into a 3D format containing the number of samples, time-steps, and features. 
This is because the decoder generates an output per time-step.
"""

## Stacked LSTM

In [ ]:
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

model = Sequential()

# encoder layer
model.add(LSTM(100, activation='relu', input_shape=(3, 1)))

# repeat vector
model.add(RepeatVector(3))

# decoder layer
model.add(LSTM(100, activation='relu', return_sequences=True))

model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')

print(model.summary())


"""
first LSTM layer is the encoder layer.we have added the repeat vector to our model. The repeat vector takes the output from the encoder and 
feeds it repeatedly as input at each time-step to the decoder. For instance, in the output we have three time-steps. To predict each output 
time-step, the decoder will use the value from the repeat vector, the hidden state from the previous output and the current input.

Next we have a decoder layer. Since the output is in the form of a time-step, which is a 3D format, the return_sequences for the decoder 
model has been set True. The TimeDistributed layer is used to individually predict the output for each time-step.
"""

history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1, batch_size=3)

test_input = array([300, 305, 310])
test_input = test_input.reshape((1, 3, 1))
test_output = model.predict(test_input, verbose=0)
print(test_output)

## Bidirectional LSTM

In [ ]:
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

model = Sequential()
model.add(Bidirectional(LSTM(100, activation='relu', input_shape=(3, 1))))
model.add(RepeatVector(3))
model.add(Bidirectional(LSTM(100, activation='relu', return_sequences=True)))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')

history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1, batch_size=3)

## Many-to-Many Sequence Problems with Multiple Features

## Stacked LSTM

In [ ]:
"""
in many-to-many sequence problems, each time-step in the input sample contains multiple features.
"""

X = list()
Y = list()
X1 = [x1 for x1 in range(5, 301, 5)]
X2 = [x2 for x2 in range(20, 316, 5)]
Y = [y for y in range(35, 331, 5)]

X = np.column_stack((X1, X2))

"""
In the script above we create two lists X1 and X2. The list X1 contains all the multiples of 5 from 5 to 300 (inclusive) and the list X2 
contains all the multiples of 5 from 20 to 315 (inclusive). Finally, the list Y, which happens to be the output contains all the multiples 
of 5 between 35 and 330 (inclusive). The final input list X is a column-wise merger of X1 and X2.
"""

X = np.array(X).reshape(20, 3, 2)
Y = np.array(Y).reshape(20, 3, 1)

"""
You can see the input X has been reshaped into 20 samples of three time-steps with 2 features where the output has been reshaped 
into similar dimensions but with 1 feature.

[[ 5  20]
[ 10  25]
[ 15  30]]

The input contains 6 consecutive multiples of integer 5, three each in the two columns.
[[ 35]
[ 40]
[ 45]]
"""

"""
The following script trains the stacked LSTM model. You can see that the input shape is now (3, 2) corresponding to three time-steps 
and two features in the input.
"""

from keras.layers import RepeatVector
from keras.layers import TimeDistributed

model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(3, 2)))
model.add(RepeatVector(3))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')

history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1, batch_size=3)

X1 = [300, 305, 310]
X2 = [315, 320, 325]

test_input = np.column_stack((X1, X2))

test_input = test_input.reshape((1, 3, 2))
print(test_input)

## Bidirectional LSTM

In [ ]:
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

model = Sequential()
model.add(Bidirectional(LSTM(100, activation='relu', input_shape=(3, 2))))
model.add(RepeatVector(3))
model.add(Bidirectional(LSTM(100, activation='relu', return_sequences=True)))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')

history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1, batch_size=3)

In [ ]:
encoder_inputs = Input(shape=(max_sequence_length,), name='encoder_inputs')
encoder_embedding = Embedding(input_dim=vocab_size_input, output_dim=embedding_dim, weights=[embedding_matrix_input],
                              input_length=max_sequence_length, trainable=False)(encoder_inputs)
encoder_lstm = Bidirectional(LSTM(latent_dim, return_state=True, name='encoder_lstm', dropout=0.5, recurrent_dropout=0.5))
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm(encoder_embedding)

# Concatenate forward and backward states
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

# Decoder
decoder_inputs = Input(shape=(31, y_max_sequence_length,), name='decoder_inputs')
decoder_embedding = TimeDistributed(Embedding(input_dim=vocab_size_input, output_dim=embedding_dim, weights=[embedding_matrix_input],
                                              trainable=False))(decoder_inputs)

decoder_outputs = []
for i in range(31):
    decoder_lstm = LSTM(latent_dim * 2, return_sequences=True, return_state=False, name=f'decoder_lstm_{i}', dropout=0.5, recurrent_dropout=0.5)
    decoder_lstm_output = decoder_lstm(decoder_embedding[:, i, :], initial_state=[state_h, state_c])
    decoder_dense = TimeDistributed(Dense(vocab_size_input, activation='softmax'), name=f'decoder_dense_{i}')
    decoder_outputs.append(decoder_dense(decoder_lstm_output))

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

In [ ]:
encoder_inputs = Input(shape=(max_sequence_length,), name='encoder_inputs')
encoder_embedding = Embedding(input_dim=vocab_size_input, output_dim=embedding_dim, weights=[embedding_matrix_input],
                              input_length=max_sequence_length, trainable=False)(encoder_inputs)
encoder_lstm = Bidirectional(LSTM(latent_dim, return_state=True, name='encoder_lstm', dropout=0.5, recurrent_dropout=0.5))
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm(encoder_embedding)

# Concatenate forward and backward states
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

# Decoder
decoder_inputs = Input(shape=(31, y_max_sequence_length,), name='decoder_inputs')
decoder_embedding = TimeDistributed(Embedding(input_dim=vocab_size_input, output_dim=embedding_dim, weights=[embedding_matrix_input],
                                              trainable=False))(decoder_inputs)
decoder_lstm = LSTM(latent_dim * 2, return_sequences=True, return_state=False, name=f'decoder_lstm_{i}', dropout=0.5, recurrent_dropout=0.5)
decoder_lstm_output = decoder_lstm(decoder_embedding[:, i, :], initial_state=[state_h, state_c])
decoder_dense = TimeDistributed(Dense(vocab_size_input, activation='softmax'), name=f'decoder_dense_{i}')
decoder_outputs.append(decoder_dense(decoder_lstm_output))



model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()